# Binary Image Classification
The following notebook is responsible for demonstrating how one would go about creating and evaluating a model using the two image generator.

In [1]:
import tensorflow as tf
import scipy
import numpy as np
import pandas as pd
import tensorflow.keras as keras

from matplotlib import pyplot as plt
from tensorflow.keras import datasets, layers, models
from mpl_toolkits.axes_grid1 import ImageGrid

from preprocessing.data_generator import CustomGen
from meta.paths import PATH_TO_TRAIN_PAIRS, PATH_TO_TEST_PAIRS
from models.prototype import create_model

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = (100, 100)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
EPOCHS = 1
FIT_AND_SAVE = False
BUILD_ON_PREVIOUS = False

# Create and Traing Model

In [3]:
if FIT_AND_SAVE: # creates a model from scratch and saves it
    train_pairs_df = pd.read_csv(PATH_TO_TRAIN_PAIRS) 
    train_generator = CustomGen(train_pairs_df,
                                shuffle=True, 
                                batch_size=BATCH_SIZE, 
                                image_size=IMAGE_SIZE)
    if BUILD_ON_PREVIOUS:
        model = keras.models.load_model("model.h5")
    else:
        model = create_model(IMAGE_SHAPE)
    checkpoint_filepath = "model.h5"
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        mode='max',
        save_best_only=True,
        save_freq="epoch")

    # The model weights (that are considered the best) are loaded into the model.
    model.load_weights(checkpoint_filepath)
    history = model.fit(train_generator, 
                        epochs=EPOCHS, 
                        batch_size=BATCH_SIZE,
                        callbacks=[model_checkpoint_callback])
    model.save("model.h5")
else:  # loads previous run
    model = keras.models.load_model("model.h5")

# Test Model
TODO: Only testing N pairs
TODO: Testing images are getting augmented

In [4]:
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, auc

def get_metrics(y_true, y_pred):
    pred_classifications = list(map(lambda s: 0 if s < .5 else 1, y_pred))
    ap_score = average_precision_score(y_true, y_pred)
    a_score = accuracy_score(y_true, pred_classifications)
    return ap_score, a_score

In [5]:
N_TEST_PAIRS = 500
test_pairs = pd.read_csv(PATH_TO_TEST_PAIRS).sample(n=N_TEST_PAIRS)
test_generator = CustomGen(test_pairs, shuffle=True, batch_size=BATCH_SIZE, image_size=IMAGE_SIZE, use_plain_generator=True)
predictions = model.predict(test_generator)

Found 500 validated image filenames belonging to 2 classes.
Found 500 validated image filenames belonging to 2 classes.


In [6]:
y_true = test_pairs["class_label"]
ap_score, a_score = get_metrics(y_true, predictions)

print("Test Accuracy: \t", a_score)
print("Test AP: \t", ap_score)

Test Accuracy: 	 0.5
Test AP: 	 0.2990978426754176


### Baseline Accuracy: Random Chance

In [7]:
import random
x = random.random()
random_scores = [random.random() for i in range(len(y_true))]
ap_score, a_score = get_metrics(y_true, random_scores)
print("Test Accuracy: \t", a_score)
print("Test AP: \t", ap_score)

Test Accuracy: 	 0.544
Test AP: 	 0.35767373284425996
